# Test case №1:

## Description

***We extracted data from GCP. The dataset contains 4 million rows, but we were only able to extract them in small portions, about 40,000 rows in CSV format. In total, we ended up with 110 files. The files are named as follows:***

data_000000000001.csv
data_000000000002.csv
…..
data_000000000010.csv
data_000000000011.csv
……
data_000000000100.csv
data_000000000101.csv
….
data_000000000110.csv

***Please suggest a method for uploading all these 110 files using Jupyter Notebook (or another) and merging them into a single dataset for further processing.***

The most logical step would be to first merge the data into one file. This could be done using a Python command-line utility called ***csvkit****. It can be installed as a Python module using the following command: ***pip install csvkit***. Alternatively, you can use a script in Jupyter Notebook for this purpose.

In [ ]:
import pandas as pd
import os

data = pd.DataFrame()
folder_path = 'path/csv/'

for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):
        file_path = os.path.join(folder_path, filename)
        current_data = pd.read_csv(file_path)
        data = data.append(current_data, ignore_index=True)
data.to_csv('path/data.csv', index=False)

However, since the size of the merged file will likely be very large and the available RAM may not allow for quick loading and processing of such a file, I suggest three alternatives.

## Option 1: Loading the first file, data_000000000001.csv, and analyzing its structure.

By default, for time efficiency, pandas provides approximate memory usage information for a DataFrame object. However, we are interested in precise details, so we will set the memory_usage parameter to 'deep'.

In [ ]:
import os
import time
import pandas as pd
import numpy as np
df = pd.read_csv('data_000000000001.csv',encoding='UTF')
def mem_usage(pandas_obj):
    if isinstance(pandas_obj,pd.DataFrame):
        usage_b = pandas_obj.memory_usage(deep=True).sum()
    else: 
        usage_b = pandas_obj.memory_usage(deep=True)
    usage_mb = usage_b / 1024 ** 2 
    return "{:03.2f} MB".format(usage_mb)
print (df.info(memory_usage='deep'))

### Let's analyze the data types in data_000000000001.csv.

Once we obtain this information, we will see the data types present in our DataFrame. With this knowledge, we can optimize memory usage by choosing appropriate data types for individual columns during loading. The pandas.read_csv() function has several parameters that allow us to achieve this.

The task does not specify the columns present in the dataset. Let's assume they are labeled as a, b, c, and d. For example, if a column contains dates, for the convenience of further calculations and clarity, it is better to set them as the index of the dataset.

In [ ]:
df = pd.read_csv('data_000000000001.csv', parse_dates=['b'], encoding='UTF')

Now the dates are read as the index of the dataset, and memory consumption has slightly decreased.

If there are object types in the data, let's combine the data in columns into categories where it is efficient. To determine efficiency, it's necessary to find the count of unique values in the columns, and if it is less than 50% of the total number of values in the column, combining values into categories will be effective.

In [ ]:
df_obj=df.select_dtypes(include=['object']).copy()
df_obj.describe()

Columns can be optimized in this way.

In [ ]:
for col in gl_obj.columns:
    num_unique_values = len(gl_obj[col].unique())
    num_total_values = len(gl_obj[col])
    if num_unique_values / num_total_values < 0.5:
        converted_obj.loc[:,col] = gl_obj[col].astype('category')        
    else:
        converted_obj.loc[:,col] = gl_obj[col]

To understand how much memory is being used, let's introduce a function for convenience:

In [ ]:
def mem_usage(pandas_obj):
    if isinstance(pandas_obj,pd.DataFrame):
        usage_b = pandas_obj.memory_usage(deep=True).sum()
    else: 
        usage_b = pandas_obj.memory_usage(deep=True)
    usage_mb = usage_b / 1024 ** 2 
    return "{:03.2f} MB".format(usage_mb)

And we will check whether the optimization was effective.

### Let's load all the files into one

Now that we see that the optimization has been beneficial, let's set dataset parameters right at the reading stage to immediately consider the data we are dealing with.






In [ ]:
import pandas as pd

data = pd.DataFrame()
folder_path = '/path/csv/'
files = glob.glob(folder_path + '*.csv')
for file in files:
    data = pd.read_csv(file, parse_dates=['b'], index_col='b',
                               dtype={'c': 'category', 'a': 'category', 'd': 'object'}, encoding='UTF')
    data = data.append(current_data, ignore_index=True)
data.to_csv('/path/data.csv', index=False)

In [ ]:
df = pd.read_csv('data.csv',encoding='UTF')

## Option 2: Using the chunksize parameter

If the first option is not successful, you can try using the chunksize parameter for efficient memory usage when working with large CSV files in pandas. This parameter allows processing data in chunks, avoiding loading the entire content of the file into memory at once. This approach optimally utilizes memory since the file is not loaded in its entirety, and the reading process is done line by line.

In [ ]:
chunks = pd.read_csv('large_file.csv', chunksize=chunk_size)
for chunk in chunks:
    # ...

## Option 3: Without using pandas

In this approach, files are also read line by line, and there won't be any memory overload.

In [ ]:
folder_path = '/path/csv/'
combined_data = []

for i in range(1, 111):
    file_path = f"{folder_path}data_{i:012d}.csv"
    
    with open(file_path, 'r', encoding='UTF') as file:
        if i > 1:
            next(file)
        
        for line in file:
            combined_data.append(line.strip())


with open('/path/data.csv', 'w', encoding='UTF') as combined_file:
    combined_file.write(data[0] + '\n')
    
    for line in data[1:]:
        combined_file.write(line + '\n')